#### 사용자 정의 데이터셋과 모델 & 학습
- 사용자 정의 데이터셋: iris.csv
- DNN 모델: 사용자 정의 모델
- DataSet과 DataLoader
    - DataSet=> 데이터 전처리, 턴서화 등의 작업 진행
    - DataLoader=> Dataset 인스턴스를 사용해서 배치크기 만큼 데이터를 추출함

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchmetrics.classification import F1Score
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset, DataLoader
from torchinfo import summary
import pandas as pd
from sklearn.preprocessing import LabelEncoder


In [2]:
DATAFILE='../data/iris.csv'
irisDF=pd.read_csv(DATAFILE)


In [3]:
encoder=LabelEncoder()
encoder.fit(irisDF['variety'])
irisDF['variety']=encoder.transform(irisDF['variety'])

In [ ]:
X_train, X_test, Y_train, Y_test= train_test_split(irisDF)

[2] 사용자 정의 데이터셋 클래스 생성 <hr>

In [4]:
# 클래스 목적: 학습용 데이터셋 텐서화 및 전처리
# 클래스 이름: CustomDataset
# 부모 클래스: torch.utils.data.Dataset
# 매개변수: featureDF, TargetDF


In [5]:
class CustomDataset(Dataset):
    def __init__(self, featureDF, targetDF):
        super().__init__()
        self.featureDF=featureDF
        self.targetDF=targetDF
        self.n_rows=featureDF.shape[0]
        self.n_features=featureDF.shape[1]
        

    # 데이터의 개수 반환 메서드
    def __len__(self):
        return self.n_rows
    
    # 특정 index의 데이터와 타겟 반환 메서드 => Tensor반환!
    def __getitem__(self, idx):
        featureTS= torch.FloatTensor(self.featureDF.iloc[idx].values)
        targetTS= torch.FloatTensor(self.targetDF.iloc[idx].values)
        return featureTS, targetTS

In [6]:
def convertDataFrame(data_path, exit_header):
    ext=data_path.split('.')[-1]
    if ext == 'csv':
        return pd.read_csv(data_path, header=0 if exit_header else None)
    elif ext == 'json':
        return pd.read_json(data_path,  header=0 if exit_header else None)
    elif ext in ['xlsx', 'xls']:
        return pd.read_excel(data_path,  header=0 if exit_header else None)
    else:
        return pd.read_table(data_path, header=0 if exit_header else None)

In [7]:
# 파일기반 데이터셋일 경우
# 클래스이름: FileDataset
# 매개변수: data_path, 
# 부모 클래스: utils.data.Dataset
# ___________________________________________________________________________________________
class FileDataset(Dataset):
    def __init__(self, data_path):
        super().__init__()
        #확장자 추출
        dataDF=convertDataFrame(data_path)
        self.featuresDF=dataDF.colums[:-1]
        self.targetDF= dataDF[dataDF.columns[-1]]
        
        self.n_features=self.featuresDF.shape[0]
        self.n_rows= self.featuresDF.shape[1]


    # 데이터의 개수 반환 메서드
    def __len__(self):
        return self.n_rows
    
    # 특정 index의 데이터와 타겟 반환 메서드 => Tensor반환!
    def __getitem__(self, idx):
        featureTS= torch.FloatTensor(self.featureDF.iloc[idx].values)
        targetTS= torch.FloatTensor(self.targetDF.iloc[idx].values)
        return featureTS, targetTS

[3] 데이터셋 인스턴스 생성 <hr>

In [8]:
featureDF, targetDF= irisDF[irisDF.columns[:-1]], irisDF[[irisDF.columns[-1]]]
print(targetDF, featureDF.shape)
irisDF= CustomDataset(featureDF, targetDF)

     variety
0          0
1          0
2          0
3          0
4          0
..       ...
145        2
146        2
147        2
148        2
149        2

[150 rows x 1 columns] (150, 4)


In [9]:

irisDF.n_features,irisDF.n_rows,

(4, 150)

[4] 데이터로더 인스턴스 생성


In [10]:
# 필요한 것: Dataset 인스턴스, batch_size=1(기본)
irisDL=DataLoader(irisDF, batch_size=10, shuffle=True)    #iterable

[5] 모델 준비

In [11]:
# 모델 이름: CustomModel
# 부모 클래스: nn.Module
# 매개변수: in_in, in_out, h_inout, h_cnt
# 모델 구조
# - 입력층: 입력 4개,  출력 10개, AF ReLU
# - 은닉층: 입력 10개, 출력 30개, AF ReLU
# - 출력층: 입력 30개, 출력 1개, AF X, softmax(다중분류)
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.in_layer= nn.Linear(4,10)
        self.hidden_layer= nn.Linear(10,30)
        self.out_layer= nn.Linear(30,1)
    # 순방향 학습
    def forward(self,x):
        y= F.relu(self.in_layer(x))
        y= F.relu(self.hidden_layer(y))
        return self.out_layer(y)
    

In [12]:
#모델 인스턴스 생성
model= CustomModel()
summary(model, input_size=(2,4))


Layer (type:depth-idx)                   Output Shape              Param #
CustomModel                              [2, 1]                    --
├─Linear: 1-1                            [2, 10]                   50
├─Linear: 1-2                            [2, 30]                   330
├─Linear: 1-3                            [2, 1]                    31
Total params: 411
Trainable params: 411
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

In [14]:
# 검증
def testing(model, X_val, Y_val):
    X_val= torch.FloatTensor(X_val.values).to(DEVICE)
    Y_val= torch.FloatTensor(Y_val.values).reshape(-1,1).to(DEVICE)
    with torch.no_grad():
        
        pre_y= model(X_val)

        loss=nn.CrossEntropyLoss()(pre_y, Y_val)

        score=F1Score()(pre_y, Y_val)
        return loss, score

In [48]:
# 배치크기만큼 데이터와 타겟 추출해서 학습 진행
loss_hist=[]
score=[]
for dataTS, targetTS in irisDL:  #1에포크
    # print(dataTS) #확인용
    # 학습
    pre_y= model(dataTS)

    # 손실계산
    loss= nn.CrossEntropyLoss()(pre_y, targetTS)
    loss_hist.append(loss)

    # 최적화
    
    
    
    

tensor(-0., grad_fn=<DivBackward1>)
tensor(-0., grad_fn=<DivBackward1>)
tensor(-0., grad_fn=<DivBackward1>)
tensor(-0., grad_fn=<DivBackward1>)
tensor(-0., grad_fn=<DivBackward1>)
tensor(-0., grad_fn=<DivBackward1>)
tensor(-0., grad_fn=<DivBackward1>)
tensor(-0., grad_fn=<DivBackward1>)
tensor(-0., grad_fn=<DivBackward1>)
tensor(-0., grad_fn=<DivBackward1>)
tensor(-0., grad_fn=<DivBackward1>)
tensor(-0., grad_fn=<DivBackward1>)
tensor(-0., grad_fn=<DivBackward1>)
tensor(-0., grad_fn=<DivBackward1>)
tensor(-0., grad_fn=<DivBackward1>)
